In [ ]:
from cil.framework import AcquisitionGeometry, ImageGeometry
from cil.io import NEXUSDataWriter
from cil.plugins.astra.processors import FBP
from cil.utilities.display import show2D
from utils import download_zenodo

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import time

import os

**First we need to download the raw data files used for reconstruction from [Zenodo](https://zenodo.org/record/5825464). For the powder phantom, there are three main datasets:**

1) powder_phantom_180s_sinogram.mat (Matlab file for Scan A dataset of 180 projections, 180s exposure time. The dataset has already been flatfield corrected).

2) powder_phantom_30s_sinogram.mat (Matlab file for Scan B dataset of 30 projections, 30s exposure time. The dataset has already been flatfield corrected).

3) Energy_axis.mat (Matlab file providing the direct energy-channel conversion, useful for analysing reconstructed datasets at different channels or different energies).

This may take some time.  

**Note:** The `download_zenodo` function requires the `wget` python package to access Zenodo files. If you don't have it, you can install using the command `conda install -c conda-forge python-wget`.

**Note 2:** You can skip this part if you already downloaded the powder data from the accompanying script `Powder_Phantom_30s_30Proj_FDK_TVTGV.ipynb`.

In [ ]:
download_zenodo()

In [ ]:
#%% Read data for Scan A datasets
pathname = os.path.abspath("MatlabData/")
      
# Scan A dataset - 180s, 180 projections
datafile = "Powder_phantom_180s_180Proj_sinogram.mat"

path = os.path.join(pathname,datafile)

tmp_X = sio.loadmat(path)   
X = tmp_X['S_180_180']

# Read Energy-Channel conversion
tmp_energy_channels = sio.loadmat(pathname + "/Energy_axis.mat")
ekeV = tmp_energy_channels['E_axis']
ekeV_crop = ekeV[0][99:199]

Sinogram raw data shape is [Vertical, Angles, Horizontal, Channels].  
However we need it in the shape [Channels, Vertical, Angles, Horizontal].  
We reorder using `np.swapaxes`

In [ ]:
print('Original Shape: {}'.format(X.shape))
X = np.swapaxes(X, 0, 3)
X = np.swapaxes(X, 1, 2)
print('Reordered Shape: {}'.format(X.shape))

In [ ]:
#%% Crop and rotate data to match data in paper

X = X[99:199] # Crop data to reduced channel subset (channels 100-200)
X = np.transpose(X,(0,3,2,1)) # Rotate data
print('Reduced Shape: {}'.format(X.shape))

In [ ]:
#%% Data shape information
num_channels = X.shape[0]
horizontal = X.shape[3]
vertical = X.shape[1]
num_angles = X.shape[2]

angles = np.linspace(-180+45,180+45,num_angles,endpoint=False)*np.pi/180

In [ ]:
#%% Define imaging scan metadata

# Scan parameters
distance_source_center = 318.0  # [mm]
distance_center_detector = 492.0  # [mm]
detector_pixel_size = 0.25  # [mm]

In [ ]:
#%% Define AcquisitionGeometry from imaging scan parameters

ag = AcquisitionGeometry.create_Cone3D(source_position = [0,-distance_source_center,0],
                                       detector_position = [0,distance_center_detector,0])\
                                     .set_panel([horizontal,vertical],[detector_pixel_size,detector_pixel_size])\
                                     .set_channels(num_channels)\
                                     .set_angles(-angles,angle_unit="radian")\
                                     .set_labels(['channel', 'vertical', 'angle', 'horizontal'])

# Create the 4D acquisition data
data = ag.allocate()
data.fill(X)

print(data)

In [ ]:
# Get the ImageGeometry directly from the AcquisitionGeometry using ig = ag.get_ImageGeometry()

ig = ag.get_ImageGeometry()

In [ ]:
#%% Plot Sinogram over different energies

from mpl_toolkits.axes_grid1 import AxesGrid

recons = [X[0,40,:,:], 
          X[25,40,:,:],
          X[50,40,:,:], 
          X[75,40,:,:]]

labels_text = ['{:.2f} keV'.format(ekeV_crop[0]), '{:.2f} keV'.format(ekeV_crop[25]),
               '{:.2f} keV'.format(ekeV_crop[50]), '{:.2f} keV'.format(ekeV_crop[75])]

# set fontsize xticks/yticks
plt.rcParams['xtick.labelsize']=15
plt.rcParams['ytick.labelsize']=15

fig = plt.figure(figsize=(12, 12))

grid = AxesGrid(fig, 111,
                nrows_ncols=(1, 4),
                axes_pad=0.05,
                cbar_mode='single',
                cbar_location='right',
                cbar_size = 0.5,
                cbar_pad=0.1
                )

k = 0

for ax in grid:
    im = ax.imshow(recons[k], cmap="inferno", vmin = 0.0, vmax = 3.0)   
    
    if k==0:
        ax.set_title(labels_text[0],fontsize=20)
    if k==1:
        ax.set_title(labels_text[1],fontsize=20)  
    if k==2:
        ax.set_title(labels_text[2],fontsize=20)  
    if k==3:
        ax.set_title(labels_text[3],fontsize=20)  
    
    ax.set_xticks([])
    ax.set_yticks([])
    k+=1

cbar = grid.cbar_axes[0].colorbar(im,ticks=[0.0,0.5,1,1.5,2,2.5,3])

In [ ]:
# Setup the tomography operator for 3D hyperspectral data using the AcquisitionGeometry and ImageGeometry

ag3D = ag.get_slice(channel=0)
ig3D = ag3D.get_ImageGeometry()

## FDK Reconstruction

In [ ]:
# Allocate space for the FBP_4D recon

FBP_recon_4D = ig.allocate()

t = time.time()

# FBP reconstruction per channel
for i in range(ig.channels):
    
    FBP_recon_3D = FBP(ig3D, ag3D, 'gpu')(data.get_slice(channel=i))
    FBP_recon_4D.fill(FBP_recon_3D, channel=i)
    
    print("Finish FBP recon for channel {}".format(i), end='\r')
    
print("\nFDK Reconstruction Complete!")
tot = time.time() - t
print('Runtime: {} s'.format(tot))

In [ ]:
#Test image

plt.imshow(FBP_recon_4D.as_array()[50,40,:,:],cmap='inferno')

In [ ]:
#%% Save as nxs file with NEXUSDataWriter

name = "Powder_180s_180Proj_FDK.nxs"
writer = NEXUSDataWriter(file_name = "HyperspectralData/" + name,
                         data = FBP_recon_4D)
writer.write()